<a href="https://colab.research.google.com/github/hoonzi-s/hongong_MLDL/blob/main/05-2.%20%EA%B5%90%EC%B0%A8%20%EA%B2%80%EC%A6%9D%EA%B3%BC%20%EA%B7%B8%EB%A6%AC%EB%93%9C%20%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검증 세트

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split as tts
train_input, test_input, train_target, test_target = tts(data, target, test_size = 0.2, random_state = 42)

In [4]:
sub_input, val_input, sub_target, val_target = tts(train_input, train_target, test_size = 0.2, random_state = 42)

In [5]:
print(data.shape, train_input.shape, sub_input.shape, val_input.shape, test_input.shape)

(6497, 3) (5197, 3) (4157, 3) (1040, 3) (1300, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증

In [7]:
# k-폴드 교차 검증
# 보통 5-폴드, 10-폴드 교차 검증을 많이 사용
# 데이터의 80~90%까지 훈련에 사용 가능

In [8]:
# cross_validate() 교차 검증 함수 (기본적으로 5-폴드)
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.01079535, 0.0071578 , 0.00751424, 0.00746751, 0.00752425]),
 'score_time': array([0.00087142, 0.00072956, 0.00075006, 0.00134039, 0.00083065]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [10]:
# 교차 검증의 최종 점수 구하기
import numpy as np
np.mean(scores['test_score'])

0.855300214703487

In [12]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

In [15]:
# 훈련 세트 섞은 후 10-폴드 교차 검증
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
np.mean(scores['test_score'])

0.8574181117533719

# 하이퍼파라미터 튜닝

In [21]:
# 사이킷런의 GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한번에 수행

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
params

{'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)

In [23]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [24]:
# GreadSearchCV의 cv 매개변수 기본값은 5.
# 즉 min_impurity_decrease 값마다 5-폴드 교차 검증 --> 총 25개 모델 훈련

In [25]:
# best_estimator_ 속성에 검증 점수가 가장 높은 모델의 매개변수 조합이 저장되어있음
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9615162593804117

In [26]:
# best_params_ 속성에 최적의 매개변수 저장되어있음
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [27]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 mean_test_score 키에 저장되어있음
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [30]:
# 넘파이 argmax() 함수 사용하면 가장 큰 값의 인덱스 추출 가능
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}